<img src="img/bigsem.png" width="40%" align="right">
<img src="img/logo_wiwi.png" width="20%" align="left">





<br><br><br><br>

# Dynamic Programming Models in Combinatorial Optimization
**Winter Term 2022/23**


# 2. Decision Diagrams

<img src="img/decision_analytics_logo.png" width="17%" align="right">


<br>

<br>
<br>

**J-Prof. Dr. Michael Römer |  Decision Analytics Group**
                                                    


In [12]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numba import njit
from typing import NamedTuple, Callable
from dataclasses import dataclass, field
from numba.experimental import jitclass

# Overview
- Review, generic DP models and algorithms
- Exact Decision Diagrams
- REducing Exact Decision Diagrams
- Restricted Decision Diagrams
- Branch and Bound based on Decision Diagrams

## Example: the 0/1 knapsack problem

Given 
- a knapsack with a capacity $W$ 
- and a set of items, each with a weight $w_i$ and a value $p_i$
- determine the the subset of the items to put in the knapsack such that
  - the total value of the items in the knapsack is maximal and
  - the total weight of the items in the knapsack does not exceed $W$

**Example:**

<img src="./img/greedy/07.png" width="20%" align="right">

Assume you are a thief and you are about to steal the three items depicted below from an appartment. However, your backpack can only fit 35 lbs. Which items should you take?



<img src="./img/greedy/08.png" width="40%" align="left ">

## A Greedy Approach for the Knapsack Problem

- start with some item: If it (still) fits in the backpack, put it in the backpack
- repeat for the remaining items

..you never take out an item once it has been packed in the knapsack

#### In Python:

In [13]:
def greedy_knapsack(values, weights, capacity):
    solution = [] # solution array
    obj_val = 0 # accumulated objective
    total_weight = 0 # accumulated weight
    
    for i, weight in enumerate(weights): 
        if total_weight + weight <= capacity: ## if the item still fits..
            solution.append(i) ## add it and 
            total_weight+= weight # update the accumulated weight
            obj_val += values[i] # as well as the optimal objective value
    return obj_val, solution

..let us try it!

In [14]:
values = [3000,2000,1500]
weights = [30,20,15]
capacity = 35

greedy_knapsack(values, weights, capacity)

(3000, [0])

## Let us try larger instances

..there are many instance sets for the 0/1 KP
- as an example, there are some instances from D. Pisinger, see the instances folder in the repository associated with this notebook
- on the following website, you will find optimal objective function values:

http://artemisa.unicauca.edu.co/~johnyortega/instances_01_KP/

.. you find some instances in the GitHub repository in which this notebook resides
- if you download the zip with this notebook (or clone the repository), you will have them in the folder `problems/knapsack/instances`


## Reading in the instances

The following function reads an instance file

In [15]:
def read_knapsack_instance(filename):
    weights=[]
    values=[]
    with open(filename) as f: # open the file
        line = f.readline().split()  # split first row
        number_of_items = int(line[0]) # read number of items
        capacity = int(line[1]) # read capacity
        for i in range(number_of_items): # read rows for the items
            line = f.readline().split() # split row
            values.append(int(line[0])) # read value
            weights.append(int(line[1])) # read weight
    return np.array(values), np.array(weights), capacity

... let us try with a 5000-item instance and solve:

In [16]:
filename = "./../problems/knapsack/instances/knapPI_1_5000_1000_1" # optimal value: 276457 
values, weights, capacity  = read_knapsack_instance(filename)

obj_value, _ = greedy_knapsack(values, weights, capacity)
obj_value


33727

## Exercise: Improving the greedy approach by sorting items

- one way to improve the performance of this greedy algorithm for the knapsack problem is to sort items
- which sorting criteria do you consider promising?
- sort the items accordingly and try applying the greedy algorithm to the sorted items

#### Hint:
In numpy, there is the function `argsort` which does not return the sorted values of an array, but an array of the sorted sorted indexes!


In [17]:

sorted_indexes = np.argsort(-1* values/weights)

sorted_values = values[sorted_indexes]
sorted_weights = weights[sorted_indexes]

obj_value, _ = greedy_knapsack(sorted_values, sorted_weights, capacity)
obj_value


#kp_instance = KPInstance(sorted_values, sorted_weights, capacity, len(sorted_values))

276379

In [18]:
distance_matrix = np.array([
    [0,  5, 4, 10],
    [5,  0, 8,  5],
    [4,  8, 0,  3],
    [10, 5, 3,  0]
])



## The Python library `python-tsp`

see: https://github.com/fillipe-gsm/python-tsp

### offers:
- functions to read TSP instances in the tsplib-format
  
  
  

In [41]:
from python_tsp.distances import tsplib_distance_matrix

#tsplib_file = "./../problems/tsp/instances/a280.tsp" # optimal solution 2579 (lt. http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html)
tsplib_file = "./../problems/tsp/instances/brazil58.tsp" # optimal solution 25395 (lt. http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html)
#tsplib_file = "./../problems/tsp/instances/berlin52.tsp" # optimal solution  7542 (lt. http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html)

distance_matrix = tsplib_distance_matrix(tsplib_file)

#permutation, distance = tsp_nearest_neighbor(distance_matrix, [0])
#distance

- and heuristic as well as exact TSP algorithms
  - e.g. local search, simulated annealing and dynamic programming (exact: careful, may take very long)

In [42]:
from python_tsp.heuristics import solve_tsp_local_search, solve_tsp_simulated_annealing

#permutation, distance = solve_tsp_local_search(distance_matrix)

permutation, distance = solve_tsp_simulated_annealing(distance_matrix)
distance

26640

## A generic wrapper function for calling a function solving the TSP
- to simplify our further experiments let us write a function that can call any function solving a TSP
- we use `*args` to allow passing arguments (e.g. start node) to the function
- the wrapper function
  - reads the distance matrix
  - solves the problem
  - checks the solution for feasibility
  - prints instance name, function name, total distance and time spent for solving

In [43]:
import timeit
from timeit import default_timer as timer

instance_name = "brazil58"
def solve_tsp_using_function(instance_name, tsp_function, *args):
    tsplib_file = f"./../problems/tsp/instances/{instance_name}.tsp" 
    distance_matrix = tsplib_distance_matrix(tsplib_file)
    starttime = timer()  
    permutation, distance = tsp_function(distance_matrix, *args)
    if set(permutation) != set(range(len(distance_matrix))):
        print ("Not a proper permutation!")
    
    print(f"{instance_name}, {tsp_function.__name__}, distance: {distance}, time: {timer()- starttime:0.3f}")

## Trying out our generic wrapper

..with a function from the TSPLib:

In [22]:
solve_tsp_using_function(instance_name, solve_tsp_simulated_annealing)    

brazil58, solve_tsp_simulated_annealing, distance: 25757, time: 2.093


..with our nearest neighbor function

## Modeling a discrete multi-stage transition system

<img src="./img/deterministic_multistage_problem.png" width="60%">



- $k$ the current step / stage (e.g. the number of cities visited so far), out of $N$ stages.
- $x_k$ the current state needed to calculate the next step and the cost
    - e.g. the cities visited visited so far and the current city
    - the start state is defined as $x_0$
- $u_k$ a decision from the set $U_k(x_k)$ of feasible decisions when being in stage $k$ and in state $x_k$ 
  - e.g. a city that was not visited so far  
- $g(x_k, u_k)$ the cost of choosing decision $u_k$ when being in state $x_k$
  - e.g. the distance to the next city
- $f(x_k, u_k)$ a transition function that computes $x_{k+1}$ from $x_k$ und the decision $u_k$ 
  - e.g. an augmentation of the cities visited so far and an update of the current city



## A dynamic programming model

A model for such a discrete system as defined on the previous slide along with the optimization problem:

$$\min_{u_0,..,u_k,..u_{N-1}} \sum_{k=0}^{N-1} g_k(x_k,u_k)$$


..will be referred to as a **dynamic programming model** in this remainder of this course, and we will refer to this generic problem as $DP$ in what follows

#### Observe:
- here, we assume a minimization problem, but it is straightforward to obtain a corresponding maximization problem
- we also assume the cost are "stage-wise"-additive (but: they can be state-dependent!)
- we assume there are no terminal costs $g(x_N)$ (would be straightforward to include)
- there can be far more general DP models, but for now we stick to classes that can be represented as displayed above


## Representing a DP model in Python

Let us devise a Python framework for implementing a dynamic programming model.

To implement a DP model for a CO problem in this framework, we need to implement the following functions:
- a function returning the feasible decisions $U_k$ given a state $x_k$ in stage $k$ (and the data from the instance)

In [23]:
def feasible_decisions(instance, k, state):
    return

- the transition function $f(x_k, u_k)$ returning the state $x_{k+1}$ resulting from taking decision $u_k$ when being in state $x_k$

In [24]:
def transition_function(instance, k, state, decision):
    return

- the cost function

In [25]:
def cost_function(instance, k, state, decision):
    return

## Representing a DP model in Python

- we can collect these three functions in a `NamedTuple` as follows:

In [26]:
class DP(NamedTuple):
    feasible_decisions : Callable
    transition_function : Callable
    cost_function : Callable
    direction : str # 'max' or 'min'

#### States
- we are free to define our state representation
- for later purposes, it will be useful if the state variable is immutable, therefore tuples or namedtuple are useful data structures for states


#### Decisions
- in most cases in this part, we will assume that decisions are integers, but note that this is not required as long as the transition function works
- however, for now, we assume that decisions only induce a change between stages -- we will relax that requirement later in the course


#### Instance data
- all functions named above take an instance as parameter. Instance data does not have to take a certain form, it just needs to "match" the (problem-specific) functions

## Generic helper functions to deal with maximization and minimization

In [27]:
@njit 
def better(value1, value2, direction):
    if direction == "min":
        return value1 < value2
    else:
        return value1 > value2
    

In [28]:
@njit
def best_element_and_value(elements, values, direction):
    if direction == "min":
        best_index = np.argmin(values)
    else:
        best_index = np.argmax(values)
    return elements[best_index], values[best_index]

In [115]:
@njit
def get_n_best_elements_and_values(n, elements, values, direction):
   
    if direction == "min":
        sorted_indexes = np.argsort(values)
    else:
        sorted_indexes = np.argsort(-values)
    return elements[sorted_indexes[:n]], values[sorted_indexes[:n]]


In [117]:
get_n_best_elements_and_values(2, np.array([1,2,3]), np.array([2,3,5]), 'max')

(array([3, 2]), array([5, 3]))

## Example: A DP model for the Knapsack Problem
- given a  knapsack instance with $N$ items with weights $w_k$ and profits $p_k$ (zero-indexed) and capacity $W$ 

- state $x_k$: accumulated weight after adding the first $k-1$ items, $x_0 = 0$
- decision $u_k \in \{0, 1\}$ (0: do not add item $k$ to the knapsack; 1: add item $k$)
- $U_k(x_k) = \begin{cases} 
                \{0,1\} \quad \mathrm{if} \quad x_k + w_k \leq W \\
                \{0 \} \quad \mathrm{else}
\end{cases}$

- $f(x_k, u_k) = x_k + w_k u_k $

- $g(x_k, u_k) = p_k u_k$

We have a maximization-objective:

$$\max_{u_0,..,u_k,..u_{N-1}} \sum_{k=0}^{N-1} g_k(x_k,u_k)$$

## The Knapsack DP Model in Python

In [31]:
class  KPInstance(NamedTuple):
    values:np.array
    weights:np.array
    capacity:int
    N:int   

@njit
def feasible_decisions_kp(instance, k, acc_weight):    
    if acc_weight + instance.weights[k] <= instance.capacity: return [0,1]  
    else: return [0]      

@njit
def transition_function_kp(instance, k, acc_weight, put):
    return acc_weight + put*instance.weights[k]

@njit
def cost_function_kp(instance, k, acc_weight, put):
      return put*instance.values[k]

Putting all together, and stating that we have a maximization objective

In [32]:
dp_kp = DP(feasible_decisions_kp, transition_function_kp,  cost_function_kp, "max")

## An instance reader function for the Knapsack Problem

In [33]:

def read_kp_instance(filename, sorted=True):
    weights=[]
    values=[]
    with open(filename) as f: # open the file
        line = f.readline().split()  # split first row
        number_of_items = int(line[0]) # read number of items
        capacity = int(line[1]) # read capacity
        for i in range(number_of_items): # read rows for the items
            line = f.readline().split() # split row
            values.append(int(line[0])) # read value
            weights.append(int(line[1])) # read weight
            
    values = np.array(values)
    weights = np.array(weights)    
    if sorted:
        sorted_indexes = np.argsort(-1* values/weights)
    values = values[sorted_indexes]
    weights = weights[sorted_indexes]
     
        
    return KPInstance(values, weights, capacity, number_of_items)



In [34]:
#filename = "./../problems/knapsack/instances/knapPI_1_5000_1000_1"
filename = "./../problems/knapsack/instances/knapPI_1_100_1000_1"
kp_instance = read_kp_instance(filename)

## Example: A DP model for the TSP

Given:
- a TSP instance with a $N$ cities and distances $d_{i,j}$ between cities $i,j$
  - let us denote with $\mathcal{N} = \{1, \ldots N \}$ the set of cities 
  


- state $x_k$: sequence / ordered set of cities visited so far, $x_0 = i^0$ where $i^0$ is the first city
  - let us define $l(x_k)$ as the last element in the ordered set, that is, the "current" city

- decision $u_k \in \mathcal{N}$ city to visit next 
- $U_k(x_k) = \mathcal{N} \setminus x_k$
- $f(x_k, u_k) = x_k + u_k$  (here, with $+$ we mean to append $u_k$ to the sequence / ordered set $x_k$
- $g(x_k, u_k) = \begin{cases} 
                d_{l(x_k), u_k} \quad \mathrm{if} \quad k < N-1 \\
               d_{l(x_k), u_k} +  d_{u_k, i^0} \quad  \mathrm{if} \quad k = N-1
\end{cases}$

We have a minimization objective:

$$\min_{u_0,..,u_k,..u_{N-1}} \sum_{k=0}^{N-1} g_k(x_k,u_k)$$



## Example: DP model for the TSP in Python

In [93]:
class TSPInstance(NamedTuple):
    distance_matrix : np.array
    N : int

@njit
def feasible_decisions_tsp(instance, k, sequence):
    return np.array([i for i in range(instance.N) if i not in sequence ])    

@njit
def transition_function_tsp(instance, k, sequence, neighbor):
    return sequence + [neighbor]

@njit
def cost_function_tsp(instance, k, sequence, neighbor):
    
    if k < instance.N-1:
        return instance.distance_matrix[sequence[k-1]][neighbor]
    else:
        return instance.distance_matrix[sequence[k-1]][neighbor] + instance.distance_matrix[neighbor][sequence[0]]

dp_tsp = DP(feasible_decisions_tsp, transition_function_tsp,  cost_function_tsp, "min")

## An instance reader function for the TSP

In [36]:
def read_tsp_instance(filename):
    
    distance_matrix = tsplib_distance_matrix(filename)
    return TSPInstance(distance_matrix, len(distance_matrix))



In [45]:
tsplib_file = "./../problems/tsp/instances/brazil58.tsp" # optimal solution 25395 (lt. http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html)
#tsplib_file = "./../problems/tsp/instances/berlin52.tsp" # optimal solution  7542 (lt. http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html)

tsp_instance = read_tsp_instance(tsplib_file)

# Generic implentations of algorithms based on the generic DP model

## Generic myopic greedy for DP models

- given a generic DP model, we can now start devising generic implementation of algorithms operating on DP models
- as an example, we can generically implement greedy as follows:

**Observe:**
- below, we avoid some loops by directly working on arrays of decisions


In [47]:
@njit
def dp_greedy(dp, instance, k_start, state_start):
    
    state = state_start
    
    total_cost = 0
    
    for k in range(k_start, instance.N):    
        
        # get decisions and associated costs
        decisions = dp.feasible_decisions(instance, k, state)               
        costs = np.array([dp.cost_function(instance, k, state, d) for d in decisions])
        
        #get the best decision according to one-step costs
        best_decision, best_cost = best_element_and_value(decisions, costs, dp.direction) 
        
        state = dp.transition_function(instance, k, state, best_decision)

        total_cost += best_cost
        
    return  state, total_cost

## Try with both KP and TSP:

Knapsack Problem:

In [54]:
dp_greedy(dp_kp, kp_instance, 0, 0)[1]

8817

TSP:

In [53]:
dp_greedy(dp_tsp, tsp_instance, 1, [0])[1]

30774

## Greedy as a myopic policy 

- we will see later how to solve a $DP$ model to optimality

- in general, we refer to a function $\pi$ that maps a state $x_k$ to a decision $u_k$ as a **policy**
- in a deterministic problem, given a policy $\pi$, we can obtain a solution to $DP$ by 
  - starting from $x_k := x_0$ and selecting the $u_k$ according to the policy
  - applying the state transition $x_{k+1} = f(x_k, u_k)$
  - and continue until $k:= N -1$

Greedy as a policy:
- we can view the greedy algorithm as being based on a policy that selects a $u_k$ that minimizes the transition costs $g$:

$$u_k = \underset{u_k \in U_k(x_k)}{\operatorname{argmin}} \, g(x_k, u_k)$$

Observe: This policy is **myopic** since it does not account for how deciding for a certain $u_k$ affects the quality of the remaining solution process - hence its name: *greedy*.

## Accounting for the future: The value function 


- to quantify the future value (also called cost-to-go) of a state $x_k$, we use the so-called value function $J(x_k)$ 
  - given a state $x_k$, $J(x_k)$ represents the cost / value obtained by solving the residual problem from stages $k$ to $N-1$.
  - the corresponding problem starting at $k$ is also referred to as the **tail subproblem**.
  
 

 
   
Given a value function, we can compute the decision to take in stage  $k$ as:

$$u_k = \underset{u_k \in U_k(x_k)}{\operatorname{argmin}} \, \Big( g(x_k, u_k) + J(f(x_k, u_k)) \Big) $$


Observe that for the greedy policy for the knapsack, $J(x_{k}) = 0$ 

## Q-values / Q-factors

- in some cases, in particular in some reinforcement learning approaches, it is convenient to use so-called Q-factors,  Q-values or Q-functions

$$Q_k(x_k, u_k) = g(x_k, u_k) + J\big( f(x_k, u_k ) \big)$$

..using these Q-factors, we can re-write the problem of selecting the next decision / control / action as:

$$u_k = \underset{u_k \in U_k(x_k)}{\operatorname{argmin}} \, Q_k(x_k, u_k) $$

## Optimal and approximate value functions


#### The optimal (exact) value function
- we denote the exact / optimal value function with $J^*$.
- if we have access to $J^*$, then the greedy policy based on $g(x_k, u_k) + J(f(x_k, u_k))$ gives us an optimal solution
- the problem: $J^*$ is typically probitively hard

#### Approximate value functions

- we denote an approximate value function with $\tilde{J}$
- a greedy policy based on $\tilde{J}$ is suboptimal,  but can be much faster to compute
- approximate value functions can be determined in various ways $\tilde{J}$:
  - using offline training / learning
  - using problem simplification or aggregation (solve an approximate tail problem)
  - using online techniques (e.g. rollout), see later

## Generic Rollout with a Base Heuristic

- recall that given an approximate value function $\tilde{J}$, we can construct a policy that takes the decision according to the best approximate $Q-$-value $\tilde{Q}_k(x_k, u_k)$

$$u_k = \underset{u_k \in U_k(x_k)}{\operatorname{argmin}} \, \tilde{Q}_k(x_k, u_k) = \underset{u_k \in U_k(x_k)}{\operatorname{argmin}} \, \Big( g(x_k, u_k) +  \tilde{J}(f(x_k, u_k)) \Big) $$
  
  
- key idea of rollout:  run a (simple and fast) base heuristic on the tail subproblem starting from $x_{k+1} = f(x_k, u_k)$ to obtain a cost / value $H(f(x_k, u_k))$, and use that value as value function approximation:
  - $\tilde{J}(x_{k+1}) = H (x_{k+1})$


<img src="./img/rollout_general.png" width="60%">

## Generic rollout with greedy base heuristic in Python
- given that we have a generic greedy, we can also implement a generic rollout algorithm
- observe: by using `@njit(parallel=True)`, numba will try parallelzing the list comprehension involving the dp_greedy call

In [88]:
@njit(parallel=True)
def dp_rollout(dp, instance, k_start, state_start):
    
    state = state_start
    
    total_cost = 0
    
    for k in range(k_start, instance.N):    
         
        decisions = dp.feasible_decisions(instance, k, state)        
        costs = np.array([dp.cost_function(instance, k, state, d) for d in decisions])        
    
        next_states = [dp.transition_function(instance, k, state, d) for d in decisions]
        
        # here, we now rund the generic greedy
        approx_costs_to_go = np.array([dp_greedy(dp, instance, k+1, next_state)[1] for next_state in next_states])
        
        # now, decision is made based on one-step cost + cost-to-go-approximation
        best_decision,_ = best_element_and_value(decisions, costs+approx_costs_to_go, dp.direction)
        
        state = dp.transition_function(instance, k, state, best_decision)

        total_cost += dp.cost_function(instance, k, state, best_decision)
        
    return state, total_cost


## Trying it out

In [72]:
%%time
dp_rollout(dp_kp, kp_instance,0,0)[1]

CPU times: total: 781 ms
Wall time: 608 ms


8929

In [131]:
%%time
dp_rollout(dp_tsp, tsp_instance,1,[0])[1]

CPU times: total: 141 ms
Wall time: 138 ms


28131

## Generic simplified rollout

- recall that, just as Bertsekas in his books, use the term `simplified` to state that only a promising subset of all decisions in a given stage will be evaluated using the greedy heuristic

In [126]:
@njit(parallel=True)
def dp_simplified_rollout(dp, instance, k_start, state_start, n_candidates):
    
    state = state_start
    
    total_cost = 0
    
    for k in range(k_start, instance.N):    
        
        
        decisions = np.array(dp.feasible_decisions(instance, k, state))
        
        costs = np.array([dp.cost_function(instance, k, state, d) for d in decisions])
        
        
        # simplification step!
        decisions, costs = get_n_best_elements_and_values(n_candidates, decisions, costs, dp.direction)
        
        
        next_states = [dp.transition_function(instance, k, state, d) for d in decisions]
        
        approx_costs_to_go = np.array([dp_greedy(dp, instance, k+1, next_state)[1] for next_state in next_states])
        
        best_decision,_ = best_element_and_value(decisions, costs+approx_costs_to_go, dp.direction)
        
        state = dp.transition_function(instance, k, state, best_decision)

        total_cost += dp.cost_function(instance, k, state, best_decision)
        
    return state, total_cost



## Trying it out

In [127]:
%%time
dp_simplified_rollout(dp_kp, kp_instance, 0, 0, 10)[1]

CPU times: total: 828 ms
Wall time: 759 ms


8929

In [ ]:
%%time
dp_simplified_rollout(dp_tsp, tsp_instance, 1, [0], 10)[1]

## Multi-step lookahead

- in (exact) dynamic programming (by reaching), we (somewhat) construct a full state-transition graph
- in rollout, in each iteration, only the first step is "exact", the rest of the graph is approximated
- in multi-step lookahead, we partially expand the tree for (more than one stage) to have more "exact" steps before using a value function approximation for selection
- below: multi-step lookahead with rollout for value function approximation

<img src="./img/multistep_lookahead.png" width="60%">

## Generic Simplified Multistage Lookahead

In [129]:
@njit
def dp_simplified_multi_stage_lookahead_rollout(dp, instance, k_start, state_start, n_candidates, number_of_lookahead_steps = 1):
    
    state = state_start
    
    total_cost = 0
    
    for k in range(k_start, instance.N):    
        
        
        decisions = np.array(dp.feasible_decisions(instance, k, state))
        
        costs = np.array([dp.cost_function(instance, k, state, d) for d in decisions])
        
        # simplification step
        decisions, costs = get_n_best_elements_and_values(n_candidates, decisions, costs, dp.direction)
        
        next_states = [dp.transition_function(instance, k, state, d) for d in decisions]
                
        if number_of_lookahead_steps == 1: #this is basically rollout
            
            approx_costs_to_go = np.array([dp_greedy(dp, instance, k+1, next_state)[1] for next_state in next_states])
        else:
            approx_costs_to_go = np.array([dp_simplified_multi_stage_lookahead_rollout(dp, instance, k+1, next_state, n_candidates, number_of_lookahead_steps - 1)[1] for next_state in next_states])
            
            
        best_decision,_ = best_element_and_value(decisions, costs+approx_costs_to_go, dp.direction)
        
        state = dp.transition_function(instance, k, state, best_decision)

        total_cost += dp.cost_function(instance, k, state, best_decision)
        
    return state, total_cost



## Trying it out

## Task: Scheduling jobs on two identical machines with time-dependent processing times, minimizing total completion time

Let us consider the following problem:
- we are given a set $J$ of of jobs that can be processed on two identical machines, starting at period $t$ = 1
- each job has a (machine-independent) "basic" processing time of $b_j$; however, the processing time depends on the start time $t_j$ at which job $j$ is started: the processing time is then $b_jt_j$
- jobs cannot be interrupted
- we are looking for an assignment of jobs to machines such that the total completion time, that is, the sum of the total processing time on both machines is minimized

- as a toy instance, consider:

In [113]:
basic_job_durations = [4,7,9,3,8,5,2]

- for large instances, just use the knapsack instances, using the items as jobs and the weights as basic processing times, or randomly generate instances

**Task:** Write a DP model (both "on paper" and in Python) for that problem

## Solving exactly with DP by Reaching

- in all the approaches discussed so far, we constructed the solution stage-by-stage, in a single pass
- one possible exact approach is to 

- we can call the resulting graph the state-transition graph of the DP

## Exact Dynamic Programming: An Illustration of the Knapsack Case

One approach to exactly solve a DP model is to
- create the state transition graph and
- compute the shortest (longest) path in the graph


<img src="./img/reaching_05.png" width="60%">



## Decision Diagrams for Optimization: An Overview

An **exact** Decision Diagram is (almost) the same as the state space graph of a DP for a maximization problem

However, the field of DDs for optimization involve a set of interesting and generic concepts:
- an exact reduction scheme that allows reducing the size of an exact DD
- restricted DDs for compactly representing a subset of all feasible solutions and for obtaining lower bounds
- relaxed DDs compactly representing a superset of all feasible solutions and for obtaining upper bounds
- a generic branch-and-bound scheme only relying on restricted and relaxed DDs





## Exact Decision Diagrams

- given a DP model, we can view an exact DD as a state-transition-graph, with one exception:
  - we introduce a terminal node that forms the target of all arcs emanating from layer $N-1$
- just as in the DP by reaching algorithm, we can construct the exact DD by 


## A Decision Diagram data structure

We will introduce a class `DecisionDiagram` that represents a DD
- consisting of $N$ + 1 layers indexed from 0 to $N$
    - each layer is a dictionary where the key is a state and the value is a `NodeInfo` object
  - (problem-specific) state values representing the start (source) state and the sink state
  

In [46]:
class DecisionDiagram:        
    def __init__(self, number_of_layers, source_layer, source_state, sink_state, direction = 'max'):    
        self.number_of_layers = number_of_layers
        self.layers = [dict() for l in range(0, number_of_layers)]
        self.source_state = source_state
        self.sink_state = sink_state
        self.layers[source_layer][source_state] = NodeInfo()
        self.direction = direction
        self.last_exact_layer = source_layer

# Storing node information

- A `NodeInfo` object contains
  - `decisions_succ_states`: a `dict` with key `decision` and value: state (the 'out-arcs' of the node)
  - `pred_state_decisions`: a `set` of (state, decision)-tuples (the 'in-arcs') of the node
  - `best_dist`: the best distance from source to the state found so far 
  - `best_pred_state_decision`: the tuple representing the predecessor state and decision leading to the best distance
 
 

In [111]:
@dataclass # observe: we use dataclass here instead of tuple because we need mutability
class NodeInfo():
    decisions_succ_states : dict = field(default_factory=dict)
    pred_state_decisions: set = field(default_factory=set)
    best_dist : int = 0
    best_pred_state_decision: tuple = None
  

In [48]:
NodeInfo(0)

NodeInfo(decisions_succ_states=0, pred_state_decisions=set(), best_dist=0, best_pred_state_decision=None)

## Building an exact DD by top-down-compilation
- building an exact DD is basically the same as building the DP by reaching: states are "discovered" layer per layer
- by applying the transition function to each feasible decision in each state in the layer under consideration
- in the following algorithm, we store the best distance from the source / root node as well as the preceding node in each node
- this means that the best path in the DD is computed "in passing"

## Building an exact DD by top-down-compilation in Python
- observe: here, we introduce a sink state as a "dummy" state (that is otherwise not reachable)

In [112]:
def build_exact_dd(dp, instance, start_layer, start_state, sink_state):
    
    dd = DecisionDiagram(instance.N+1, start_layer, start_state, sink_state, dp.direction)
    
    state = start_state
    total_cost = 0
    
    for k in range(0,instance.N):
        
        for state, node_info in dd.layers[k].items():
            decisions = dp.feasible_decisions(instance, k, state)
            
            for decision in decisions:
                if k < instance.N -1: # if we are the final layer, point to the "sink state"
                    next_state = dp.transition_function(instance,k,state, decision)
                else:
                    next_state = -1
                
                add_transition_dd(dd, k, state, decision, next_state, dp.cost_function(instance, k, state, decision))
   
    k = instance.N-1
    
    return dd

## Creating new nodes: adding the result of a transition

In [109]:
def add_transition_dd(dd, layer_index, state, decision, result_state, cost):
    
    layer = dd.layers[layer_index]
    result_layer = dd.layers[layer_index+1] 
    
 
    result_dist = layer[state].best_dist + cost  

    node_info = result_layer.get(result_state) 

    # add new node if state does not exist in that layer
    if node_info is None: 
        node_info = NodeInfo()
        node_info.best_dist = result_dist
        node_info.best_pred_state_decision = (state, decision)
        result_layer[result_state] = node_info            
    
    else:  ## check if betterpath 
        if better(result_dist, node_info.best_dist, dd.direction): 
            node_info.best_pred_state_decision = (state, decision)
            node_info.best_dist = result_dist

    node_info.pred_state_decisions.add((state, decision))

    layer[state].decisions_succ_states[decision] = result_state

  

## Trying it out, and some utility functions:

In [52]:
dd = build_exact_dd(dp_kp, kp_instance, 0, 0,-1)


..getting the best objective

In [51]:
def get_best_objective (dd):
    return dd.layers[dd.number_of_layers-1][dd.sink_state].best_dist

In [53]:
get_best_objective (dd)

9147

..getting the best path

In [67]:

def get_best_path(dd):
    
    decisions = []
    
    state =  dd.sink_state
    k = dd.number_of_layers - 2
        
    while k >= 0:
       # print(k)
        state, decision = dd.layers[k+1][state].best_pred_state_decision
        #print (dd.layers[k+1])
        decisions.append(decision)
        k = k-1
        

    return dd.layers[dd.number_of_layers-1][-1].best_dist, list(reversed(decisions))


In [ ]:
get_best_path(dd)[1][:10] ## first 10 nodes

..getting the number of nodes

In [68]:
def get_number_of_nodes(dd):
    return np.sum([len(layer) for layer in dd.layers])


In [ ]:
get_number_of_nodes(dd)

## Reducing an exact DD

One of the key ideas from DDs is that very often, a DD can be compressed / reduced by merging nodes 
that 
- do not have identical (top-down) states
- but are nonetheless **equivalent** in the sense that they have the same *completions*, that is, the same set of partial solutions until the end (the solution sets of their tail subproblems are identical)

This type of equivalence can be identified by an upward-pass starting from the bottom layer $N$ to layer $0$

- in each layer $k$, two nodes are equivalent (are in the same equivalence class) if 
  - they have the same set of feasible decisions 
  - these decisions have the same costs
  - the corresponding arcs point to the same set of nodes in the subsequent layer $k+1$
- for each equivalence class, merge all nodes in that class into a single node

**Attention:** The following implementation assumes that the decision costs are state-independent. If the decision costs (the arc costs) are state-dependent, then we need to add a check for identical costs, too

## Implementing the DD reduction

In [108]:
def reduce_exact_dd(dd):
    
    #proceed from the bottom (last layer) to the top
    k = len(dd.layers)-1
    while k > 0:
        
        # a dict with key: decisions and resulting nodes (forming an equivalence class)
        #       and value: list of states falling into that class
        eq_classes = {}
    
        #1. collect equivalence classes and states/nodes in each class
        for state, node_info in dd.layers[k].items():
    
            eq_class = tuple(node_info.decisions_succ_states)            
            if eq_class not in eq_classes:
                eq_classes[eq_class] = [state]                
            else:
                eq_classes[eq_class].append(state)
        
        # 2. merge all states in each class into a single node
        for eq_class, states in eq_classes.items():            
            while len(states) > 1:
                state_remove = states.pop()
                merge_nodes(dd, k, states[0], state_remove)
        k=k-1

## Merging two nodes

In [104]:
def merge_nodes(dd, layer_index, state_orig, state_remove):
    layer = dd.layers[layer_index]
    next_layer = dd.layers[layer_index+1]
    prev_layer = dd.layers[layer_index-1]

    # 1. Keep the best distance to from the source
    if better(layer[state_remove].best_dist, layer[state_orig].best_dist, dd.direction):
            layer[state_orig].best_pred_state_decision = layer[state_remove].best_pred_state_decision
            layer[state_orig].best_dist = layer[state_remove].best_dist

    # 2. remove the in-arcs of the successors that come from the removed node
    for decision, succ_state in layer[state_remove].decisions_succ_states.items():
        next_layer[succ_state].pred_state_decisions.remove((state_remove, decision))
        if next_layer[succ_state].best_pred_state_decision == (state_remove, decision):
            next_layer[succ_state].best_pred_state_decision = (state_orig, decision)

    # 3. redirect the in-arcs from the removed node to the node to be kept
    for pred_state, decision in layer[state_remove].pred_state_decisions:
        prev_layer[pred_state].decisions_succ_states[decision] = state_orig
        layer[state_orig].pred_state_decisions.add((pred_state, decision))

    # 4. Remove the node
    layer.pop(state_remove)

## Trying it  out

In [ ]:
reduce_exact_dd(dd)

print ("reduced nodes", get_number_of_nodes(dd))

..some sanity checks

In [72]:
get_best_objective (dd)

7276

In [ ]:
get_best_path(dd)[:10]

## Restricted Decision Diagrams

- building an exact decision diagram is often not practical since it may have an exponential size
- also, the reduction requires building the exact DD beforehand
- the idea of restricted DDs is to limit the size of the DD (more precisely, the width `maxWidth` of its layers) by removing states from each layer until the maximum width is respected
- of course, this introduces an approximation, that is, the solution is no longer guaranteed to be optimal


## Building a restricted DD top-down in Python

In [73]:
def build_restricted_dd(dp, instance, start_layer, start_state, sink_state, max_width,  sort_key_function=None):
    
    dd = DecisionDiagram(instance.N+1, start_layer, start_state, sink_state, dp.direction)  
    sort_key_function = get_sort_key_function(dp)
    
    state = start_state
    total_cost = 0    
    
    for k in range(start_layer,instance.N):

         for state, node_info in dd.layers[k].items():
            decisions = dp.feasible_decisions(instance, k, state)
            
            for decision in decisions:
                if k < instance.N -1:
                    next_state = dp.transition_function(instance,k,state, decision)
                else:
                    next_state = -1
                
                add_transition_dd(dd, k, state, decision, next_state, dp.cost_function(instance, k, state, decision)) 
                
         # remove_nodes until max_width is reached
        remove_until_max_width(dd, k+1,max_width, sort_key_function)       

    return dd



## Removing all nodes in a layer until maximum width

In [74]:
def remove_until_max_width(dd, layer_index, max_width, sort_key_function):

    if len(dd.layers[layer_index]) <= max_width:
        if dd.last_exact_layer == layer_index -1: 
            dd.last_exact_layer = layer_index
        return

    sorted_nodes = sorted(dd.layers[layer_index].items(), key=sort_key_function)
    dd.layers[layer_index] = dict(sorted_nodes[0:max_width])

## Some standard sort function

In [76]:
def sort_key_max_distance(state_node):
    return state_node[1].best_dist * -1

In [77]:
def sort_key_min_distance(state_node):
    return state_node[1].best_dist

In [75]:
def get_sort_key_function_best_distance(dp):
    
    if dp.direction == 'max':
        return sort_key_max_distance
    else:
        return sort_key_min_distance

In [78]:
dd = build_restricted_dd(dp_kp, kp_instance, 0, 0,-1,5)


In [ ]:
get_best_objective (dd)

## Task: Other sort functions

**Try out other sort functions!**
- using min distance instead of max distance for the KP#
- write your own function!

## Relaxed Decision Diagrams

- restricted DDs have a **limited size**, represent a **subset** of all feasible solutions of a DP, and thus give us an approximate feasible solution

- relaxed DDs have a **limited size**, represent a **superset** of all feasible solutions of a DP, and thus give us an upper (lower) bound in case of a maximization (minimization) problem
    - how can that work?

    
**Key idea:** By merging nodes that are not equivalent!
- to obtain a proper relaxation (to make sure that we have a superset of all feasible solutions, we apply a so-called **merge operation** $\oplus$ that makes sure that 
    - the solutions (paths to the terminal) starting from the merged state $s' = s_1 \oplus s_2$  form a superset of the union of the solutions (paths to the terminal) starting from $s_1$ and $s_2$
    - and that the best objective value starting from $s_1 \oplus s_2$ is at least as good as the 
- in simplified terms, $\oplus$ is chosen in a way that $s_1 \oplus s_2$ is less restrictive that both $s_1$ and $s_2$

## Top-down compilation of relaxed DDs: Python implementation

In [79]:
def build_relaxed_dd(dp, merge_function, instance, start_layer, start_state, sink_state, max_width, sort_key_function=None):
    
    dd = DecisionDiagram(instance.N+1, start_layer, start_state, sink_state)
    
    if sort_key_function is None:
        sort_key_function = get_sort_key_function_best_distance(dp)
    
    state = start_state
    total_cost = 0
    
    
    for k in range(start_layer,instance.N):
       
        for state, node_info in dd.layers[k].items():
            decisions = dp.feasible_decisions(instance, k, state)
            
            for decision in decisions:
                if k < instance.N -1:
                    next_state = dp.transition_function(instance,k,state, decision)
                else:
                    next_state = -1
                
                add_transition_dd(dd, k, state, decision, next_state, dp.cost_function(instance, k, state, decision))
        

        merge_until_max_width(dd, merge_function, k+1, max_width, sort_key_function)
     

    return dd



## Merge until the maximum is reached

In [102]:
 def merge_until_max_width(dd, merge_function, layer_index, max_width, sort_key_function):
        
        layer = dd.layers[layer_index]
        prev_layer = dd.layers[layer_index-1]
        
        if len(layer) <= max_width:
            if dd.last_exact_layer == layer_index -1: 
                dd.last_exact_layer = layer_index
            return
        
        # 1. sort the nodes, return  a list of tuples
        sorted_nodes = sorted(dd.layers[layer_index].items(),key=sort_key_function)
        
        # 2. turn the tuples into a layer of length max_width - 1 
        dd.layers[layer_index] = dict(sorted_nodes[0:max_width-1])
        
        
        # 3. merge the remaining states into a single (relaxed) node
        
        # begin with the first state
        state, node_info = sorted_nodes[max_width-1]
        
        # 3a: compute the merged state as well as the best distance of the new node        
        for state_next, node_info_next in sorted_nodes[max_width:]:            
            state = merge_function(state, state_next) # compute the merged state            
            if better(node_info_next.best_dist, node_info.best_dist, dd.direction): #compute
                node_info.best_pred_state_decision = node_info_next.best_pred_state_decision
                node_info.best_dist = node_info_next.best_dist
            
        # 3b: make sure everything is properly linked to the predecessors
        node_info.pred_state_decisions.clear()
        
        for state_next, node_info_next in sorted_nodes[max_width-1:]:
            
            for pred_state, decision in node_info_next.pred_state_decisions:
                prev_layer[pred_state].decisions_succ_states[decision] = state
                node_info.pred_state_decisions.add((pred_state, decision))
            
           
        # 3c: finally, add the merged node to the layer: now the layer should be max_width
        dd.layers[layer_index][state] = node_info

## Trying the whole procedure

In [82]:
dd = build_relaxed_dd(dp_kp, min, kp_instance, 0, 0, -1, 200)


print ("dist", get_best_objective(dd), "nodes", get_number_of_nodes(dd))



dist 7276 nodes 630


## Task: Other sort functions

**Try out other sort functions!**
- using min distance instead of max distance for the KP#
- write your own function!

## DD-based Branch-and-Bound

- for many problems, building the full exact DD is not feasible within an acceptable amount of time
- limited-size restricted and relaxed DDs (only) provide bounds
- however, they can be used in a DD-specific branch-and-bound scheme!
- that scheme was used to successfully solve a number of combinatorial optimization problems, in some cases achieving state-of-the-art performance
- the scheme is highly parallelizable

## DD-based Branch-and-Bound: Key ideas

**Key idea I: Branching on nodes in exact cutsets**

- in every relaxed (and restricted) DD, there are some nodes that are exact
- then, in a relaxed DD, we can identify so-called **exact cutsets:**
  - a **cutset** is a subset of nodes such that all source-terminal paths pass through at least one node in that subset
  - a cutset is called **exact** if all nodes represent exact states (not "relaxed" by the merge operation)
  
- now, instead of of increasing the width in layers after the exact cutset, the branch-and-bound starts building a "new" relaxed DD for the subproblem starting from each node in the cutset
- the nodes from the cutset are considered as "open nodes" which are processed one after the other
- when creating a "new" relaxed DD, we once again obtain an exact cutset that is then added to the set of open nodes

..one of the simplest ways to obtain an exact cutset is to use the "last exact layer", that is, the last layer in which no nodes needed to be removed / merged

**Key idea II: Bounding** 
- before building the relaxed DD for the current node, we build a restricted DD for the subproblem to (hopefully) obtain a new best feasible solution
- if the relaxed DD starting from the subproblem yields a solution that is worse than that "primal" solution, its exact cutset is not added to the set of open nodes

## DD-based Branch-and-Bound: Algorithm

In [95]:
from queue import PriorityQueue


def branch_and_bound_dd(dp, merge_function, instance, start_layer, start_state, sink_state, max_width):    
    
    best_feasible_obj = 0    
      
    factor_p_queue = 1
    if dp.direction == 'min':
        factor_p_queue = -1
    
    # initialise the set of open states (we use a priority queue here)
    # we store obj-function * factor as key for determining the priority, state and layer in the queue
    open_states = PriorityQueue()
    open_states.put((0, start_state, 0))
    
    number_of_open_states_considered = 1
    
    while open_states.qsize() > 0:
        number_of_open_states_considered += 1
        dist_to_state, state, layer = open_states.get() # get node from pqueue (and remove from queue)        
        dist_to_state = dist_to_state * factor_p_queue # "re-transform" if needed

        #solve restriction for the subproblem
        dd_restricted = build_restricted_dd(dp, instance, layer, state, sink_state, max_width)        
        restriction_obj = get_best_objective(dd_restricted)

        # see if we improved the best-known feasible solution
        if dist_to_state + restriction_obj > best_feasible_obj:            
            best_feasible_obj = dist_to_state + restriction_obj
           # print ("new best incumbent", best_feasible_obj )
            
        # if the subproblem is exact, no need to continue
        if dd_restricted.last_exact_layer == instance.N:
            continue
        
        #solve relaxation for the subproblem       
        dd_relaxed = build_relaxed_dd(dp, merge_function, instance, layer, state, sink_state, max_width)        
        relaxation_obj = get_best_objective(dd_relaxed) 
        
        # bounding: if cannot improve the best feasible solution, continue
        if dist_to_state + relaxation_obj <= best_feasible_obj:
            continue
    
        ## get last_exact_layer cutset and add to open nodes
        cutset = dd_relaxed.layers[dd_relaxed.last_exact_layer]    
        for state, node_info in  cutset.items():              
            open_states.put((factor_p_queue * (dist_to_state + node_info.best_dist), state, dd_relaxed.last_exact_layer))

    print ("open states considered", number_of_open_states_considered)
    return best_feasible_obj
        
        

## Trying it out

In [132]:


#filename = "./../problems/knapsack/instances/knapPI_1_100_1000_1" # optimal value: 276457 
#values, weights, capacity  = read_knapsack_instance(filename)

kp_instance = KPInstance(values, weights,capacity,len(values))
filename = "./../problems/knapsack/instances/knapPI_1_100_1000_1"
kp_instance = read_kp_instance(filename)

N = 20
kp_instance = KPInstance(kp_instance.values[:N], kp_instance.weights[:N],kp_instance.capacity,  N)

In [99]:
%%time
branch_and_bound_dd(dp_kp, min, kp_instance, 0, 0, -1, 100)

open states considered 570
CPU times: total: 3.44 s
Wall time: 3.59 s


9147

In [100]:
%%time
dd = build_exact_dd(dp_kp, kp_instance, 0, 0,-1)
print("Size of exact DD:", get_number_of_nodes(dd))
get_best_objective(dd)

Size of exact DD: 8660
CPU times: total: 266 ms
Wall time: 272 ms


9147